# Financial Advisor Workflow Using CrewAI


In this notebook, we demonstrate how to build a financial advisory system using **CrewAI**. The system integrates multiple agents, each with a specific role, to:
1. Gather financial data from the internet.
2. Analyze the financial data.
3. Provide investment recommendations.

---
### Workflow Overview
1. Define tools to support the agents, such as web search capabilities.
2. Create three agents for data gathering, analysis, and financial advice.
3. Define tasks corresponding to these agents.
4. Set up a crew that executes the tasks sequentially.
5. Run the crew to achieve the desired outcome.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [ ]:
from crewai import Agent, Task
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from a '.env' file
load_dotenv()

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Step 2: Define the Agents

In [ ]:
from crewai import Agent

# Agent to gather financial data from the internet
content_explorer = Agent(
    role='Data Researcher',
    goal='Gather and provide latest information about the topic from the internet',
    llm='gpt-4o-mini',
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a topic. '
        'Consider you are on: ' + Today
    ),
    tools=[search_tool],
    cache=True,
    max_iter=5,
)

# Agent to analyze financial data
analyst = Agent(
    role='Data Analyst',
    goal='Consolidate financial data, stock information and provide a summary',
    llm='gpt-4o-mini',
    verbose=True,
    backstory=(
        'You are an expert in analyzing financial data and stock-related information to create a summary. '
        'Consider you are on: ' + Today
    ),
)

# Agent to provide investment recommendations
fin_expert = Agent(
    role='Financial Expert',
    goal='Considering financial analysis of a stock, make investment recommendations',
    llm='gpt-4o-mini',
    verbose=True,
    backstory=(
        'You are a financial advisor, providing investment recommendations. '
        'Consider the financial analysis and make recommendations about whether to buy a stock. '
        'Consider you are on: ' + Today
    ),
)

### Step 3: Define the Tasks

In [ ]:
from crewai import Task
from my_agents import content_explorer, analyst, fin_expert

# Task to gather company financial data
get_company_financials = Task(
    description="Get latest financial data for stock: {stock}",
    expected_output="Latest data about balance sheet, cash flow, and profit for stock {stock}",
    agent=content_explorer,
)

# Task to analyze the financial data
Anlayse = Task(
    description="Make thorough analysis based on given financial data of a stock",
    expected_output="Comprehensive analysis of a stock outlining financial health, stock valuation, and risks",
    agent=analyst,
    context=[get_company_financials],
    output_file='Analysis.txt',
)

# Task to provide investment advice
Advise = Task(
    description="Make a recommendation about investing in a stock, based on analysis provided",
    expected_output="Recommendation (Buy / No Buy) of a stock, with reasons clearly mentioned",
    agent=fin_expert,
    context=[Anlayse],
    output_file='Recommendation.txt',
)

### Step 4: Set Up the Crew

In [ ]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew(
    agents=[content_explorer, analyst, fin_expert],
    tasks=[get_company_financials, Anlayse, Advise],
    verbose=True,
    Process=Process.sequential,
    step_callback=timestamp,
)

### Step 5: Run the Crew and Observe Results

In [ ]:
# Run the crew with a specific stock
result = crew.kickoff(inputs={'stock': 'Tata Steel'})

# Print the final result
print("Final Result:", result)


---
### Conclusion
This workflow demonstrates how to integrate multiple agents to perform tasks sequentially in a financial advisory context. It showcases the power of CrewAI for building AI-driven collaborative solutions.
